In [14]:
import os
import csv

def convert_txt_file_to_csv(input_txt_path, output_dir, delimiter=','):
    """
    Reads a single .txt file from input_txt_path, converts it to .csv, and saves
    the new file inside output_dir with the same base name.

    :param input_txt_path: Full path to the input .txt file.
    :param output_dir: Directory where the .csv will be saved.
    :param delimiter: Delimiter used in the .txt file (default is comma).
    """

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Get the base filename (e.g., 'error_CL_1.txt' -> 'error_CL_1')
    base_name = os.path.basename(input_txt_path)
    file_root, _ = os.path.splitext(base_name)

    # Construct output CSV path with same base name
    output_csv_path = os.path.join(output_dir, file_root + '.csv')

    with open(input_txt_path, 'r') as txt_file, open(output_csv_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)

        # Read each line from the .txt, split by delimiter, and write to CSV
        for line in txt_file:
            row_data = line.strip().split(delimiter)
            csv_writer.writerow(row_data)



In [15]:
file_names = [
    "airfoil_cm_lists_with_trip",
    "airfoil_cm_lists",
    "efficiencies_alfa_1",
    "efficiencies_alfa_2",
    "efficiencies_alfa_with_trip",
    "efficiencies_CL_1",
    "efficiencies_CL_2",
    "efficiencies_CL_with_trip",
    "error_alfa_1",
    "error_alfa_2",
    "error_alfa_with_trip",
    "error_CL_1",
    "error_CL_2",
    "error_CL_with_trip",
    "error_cm_with_trip"
]

for file_name in file_names:
    convert_txt_file_to_csv(
        input_txt_path="results_txt/" + file_name + ".txt",
        output_dir="results_csv",
        delimiter=","
    )


In [16]:
# The goal of this cell is to gather all of airfoil names errors file in only one -> unique_airfoils.csv (I did not use the tripping errors files)
import csv

file_paths = [
    'results_csv/error_alfa_1.csv',
    'results_csv/error_alfa_2.csv',
    'results_csv/error_CL_1.csv',
    'results_csv/error_CL_2.csv'
]

unique_airfoils = set()

for file_path in file_paths:
    with open(file_path, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            # If there's at least one element in this row, that's the airfoil name
            if row:
                airfoil_name = row[0].strip()
                if airfoil_name:  # ensure it's not an empty line
                    unique_airfoils.add(airfoil_name)

# Write all unique airfoils to a new CSV
output_file_path = 'results_csv/unique_airfoils.csv'
with open(output_file_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['airfoil_name'])  # Optional header
    for airfoil in sorted(unique_airfoils):
        writer.writerow([airfoil])



In [28]:
# This cell access all of the efficiency files and deletes all of error airfoils

import csv

# --------------------------------------------------
# 1) Load the reference airfoils from unique_airfoils.csv
# --------------------------------------------------
reference_file = 'results_csv/unique_airfoils.csv'
reference_airfoils = set()

with open(reference_file, 'r', newline='', encoding='utf-8') as ref_csv:
    reader = csv.reader(ref_csv)
    
    # If unique_airfoils.csv has a header (like "airfoil_name"),
    # skip it by reading one row without storing it:
    header_row = next(reader, None)
    
    # Now read the remaining lines (airfoil names)
    for row in reader:
        # Each row is presumably one name per line
        if row:
            airfoil_name = row[0].strip()
            reference_airfoils.add(airfoil_name)

# --------------------------------------------------
# 2) Define the list of target CSV files to be cleaned
# --------------------------------------------------
file_paths_to_clean = [
    # Replace these with the paths to your "attached" files
    'results_csv/efficiencies_alfa_1.csv',
    'results_csv/efficiencies_alfa_2.csv',
    'results_csv/efficiencies_CL_1.csv',
    'results_csv/efficiencies_CL_2.csv',

    # Add as many as needed
]

# --------------------------------------------------
# 3) For each file, remove rows where the first column
#    matches an entry in reference_airfoils
# --------------------------------------------------
for file_path in file_paths_to_clean:
    new_contents = []
    
    with open(file_path, 'r', newline='', encoding='utf-8') as original_csv:
        reader = csv.reader(original_csv)
        
        # If these files have a header, read it once and store it:
        header = next(reader, None)
        
        # Optional: If you want to keep the header row, add it to new_contents
        # BUT only if you are sure the first row is indeed a header.
        # If there's no header, comment these lines out.
        if header:
            new_contents.append(header)
        
        # Now read each data row
        for row in reader:
            if not row:
                continue  # skip empty lines if present
            
            # The airfoil name is assumed to be in the first column
            airfoil_in_file = row[0].strip()
            
            # Only keep the row if the airfoil is NOT in the reference set
            if airfoil_in_file not in reference_airfoils:
                new_contents.append(row)
    
    # Overwrite the original file with the filtered data
    with open(file_path, 'w', newline='', encoding='utf-8') as cleaned_csv:
        writer = csv.writer(cleaned_csv)
        writer.writerows(new_contents)
    
    print(f"File cleaned: {file_path}")


File cleaned: results_csv/efficiencies_alfa_1.csv
File cleaned: results_csv/efficiencies_alfa_2.csv
File cleaned: results_csv/efficiencies_CL_1.csv
File cleaned: results_csv/efficiencies_CL_2.csv
